In [1]:
import json
import numpy as np
import nltk
from nltk.corpus import brown 
import torch
from datasets import load_dataset
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline 

c:\Users\91959\Desktop\Personal\projects\transformers\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('brown')
nltk.download('universal_tagset')
corpus = brown.tagged_sents(tagset='universal')
corpus #output is list of lists, with tuples of token, tag 

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\91959\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\91959\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [3]:
#tag2int dictionary
tags = {tag for sentence in corpus for _, tag in sentence}
tag2int = {tag: i for i, tag in enumerate(tags)}
tag2int

{'ADV': 0,
 'CONJ': 1,
 'VERB': 2,
 'ADJ': 3,
 'ADP': 4,
 'PRON': 5,
 'X': 6,
 'NUM': 7,
 '.': 8,
 'PRT': 9,
 'NOUN': 10,
 'DET': 11}

In [4]:
#int2tag dictionary
int2tag = {value:key for key, value in tag2int.items()}
int2tag

{0: 'ADV',
 1: 'CONJ',
 2: 'VERB',
 3: 'ADJ',
 4: 'ADP',
 5: 'PRON',
 6: 'X',
 7: 'NUM',
 8: '.',
 9: 'PRT',
 10: 'NOUN',
 11: 'DET'}

In [5]:
#Convert to json
converted_data = [{
    "inputs": [token for token, _ in sentence],
    "targets": [tag2int[tag] for _, tag in sentence]
} for sentence in corpus]

filename = './datasets/brown.json'

# Use 'with' statement to open file to ensure proper closure after writing
with open(filename, 'w') as file:
    json.dump(converted_data, file, indent=4)

In [6]:
data = load_dataset(path='json', data_files='./datasets/brown.json')
data 

Generating train split: 57340 examples [00:00, 70254.38 examples/s]


DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 57340
    })
})

In [7]:
#train test split
datasets = data['train'].train_test_split(test_size=0.3, seed=42)
datasets

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 40138
    })
    test: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 17202
    })
})

In [8]:
# casing is better for pos(Bill vs bill)
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
#function for aligning targets:
def align_targets(labels, word_ids):
    aligned_labels = []
    last_id = None
    for id in word_ids:
        if id == None:
            label = -100 #transformers(pytorch crossentropy loss) uses -100 for depicting targets that should not affect loss function 
        else:
            label = labels[id] 
        aligned_labels.append(label)
    return aligned_labels 

In [10]:
#checking target alignment:
labels = datasets['train'][0]['targets']
t = tokenizer(datasets['train'][0]['inputs'], is_split_into_words=True)
word_ids = t.word_ids()
aligned_targets = align_targets(labels, word_ids)
aligned_labels =  [int2tag[t] if t>=0 else None for t in aligned_targets]
for x, y in zip(t.tokens(), aligned_labels):
    print(f"{x}\t{y}")

[CLS]	None
Co	NOUN
##st	NOUN
of	ADP
power	NOUN
and	CONJ
machinery	NOUN
is	VERB
often	ADV
a	DET
serious	ADJ
problem	NOUN
to	ADP
the	DET
small	NOUN
-	NOUN
scale	NOUN
farmer	NOUN
.	.
[SEP]	None


In [11]:
for x, y in zip(datasets['train'][0]['inputs'], datasets['train'][0]['targets']):
    print(f"{x}\t{int2tag[y]}")

Cost	NOUN
of	ADP
power	NOUN
and	CONJ
machinery	NOUN
is	VERB
often	ADV
a	DET
serious	ADJ
problem	NOUN
to	ADP
the	DET
small-scale	NOUN
farmer	NOUN
.	.


In [12]:
#tokenize fn:
def tokenize_fn(batch):
    tokenized_inputs = tokenizer(
        batch['inputs'],
        truncation=True,
        is_split_into_words=True
    )
    labels_batch = batch['targets']
    aligned_labels_batch = []
    for i, labels in enumerate(labels_batch):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels_batch.append(align_targets(labels, word_ids))
    tokenized_inputs['labels'] = aligned_labels_batch
    return tokenized_inputs

In [13]:
datasets['train'].column_names

['inputs', 'targets']

In [14]:
tokenized_datasets = datasets.map(
    function=tokenize_fn,
    batched=True,
    remove_columns=datasets['train'].column_names
)

Map: 100%|██████████| 17202/17202 [00:01<00:00, 9765.89 examples/s] 


In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 40138
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 17202
    })
})

In [16]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [17]:
#using data collator explicitly:
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch #even padding tokens have -100 as target

{'input_ids': tensor([[  101,  3291,  2050,  1104,  1540,  1105, 11360,  1110,  1510,   170,
          3021,  2463,  1106,  1103,  1353,   118,  3418,  9230,   119,   102],
        [  101, 17323, 17941,   136,   136,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[-100,   10,   10,    4,   10,    1,   10,    2,    0,   11,    3,   10,
            4,   11,   10,   10,   10,   10,    8, -100],
        [-100,    2,   10,    8,    8, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]])}

In [18]:
#trying out sklearn classification_report
classification_report(
    y_true=[0, 0, 1],
    y_pred=[0, 1, 1],
    output_dict=True
)

{'0': {'precision': 1.0,
  'recall': 0.5,
  'f1-score': 0.6666666666666666,
  'support': 2.0},
 '1': {'precision': 0.5,
  'recall': 1.0,
  'f1-score': 0.6666666666666666,
  'support': 1.0},
 'accuracy': 0.6666666666666666,
 'macro avg': {'precision': 0.75,
  'recall': 0.75,
  'f1-score': 0.6666666666666666,
  'support': 3.0},
 'weighted avg': {'precision': 0.8333333333333334,
  'recall': 0.6666666666666666,
  'f1-score': 0.6666666666666666,
  'support': 3.0}}

In [19]:
def flatten(list_of_lists):
    return [item for row in list_of_lists for item in row]

flatten([[1,2,3], [4, 5]])

[1, 2, 3, 4, 5]

In [20]:
def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis=-1)
    labels_jagged = [[l for l in row if l != -100] for row in labels]
    predictions_jagged = [[p for (p,l) in zip(ps, ls) if l !=-100] for ps, ls in zip(predictions, labels)]
    labels_flat = [label for row in labels_jagged for label in row]
    predictions_flat = [prediction for row in predictions_jagged for prediction in row]
    return classification_report(
        y_true=labels_flat, 
        y_pred=predictions_flat,  
        target_names=int2tag.values(),
        output_dict=True
        )

In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=int2tag,
    label2id=tag2int
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
training_args = TrainingArguments(
    output_dir='output_dir',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [24]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)
trainer.train()

  3%|▎         | 501/15054 [00:54<26:31,  9.14it/s]

{'loss': 0.2972, 'learning_rate': 1.9335724724325762e-05, 'epoch': 0.1}


  7%|▋         | 1002/15054 [01:49<25:00,  9.36it/s]

{'loss': 0.0743, 'learning_rate': 1.8671449448651522e-05, 'epoch': 0.2}


 10%|▉         | 1501/15054 [02:47<22:57,  9.84it/s]

...

                                                     
100%|██████████| 15054/15054 [34:20<00:00,  8.30it/s]

{'eval_loss': 0.04342537000775337, 'eval_ADV': {'precision': 0.9765848394928514, 'recall': 0.9729105079279764, 'f1-score': 0.974744211093161, 'support': 18605.0}, 'eval_CONJ': {'precision': 0.997117903930131, 'recall': 0.9975535168195718, 'f1-score': 0.9973356628084734, 'support': 11445.0}, 'eval_VERB': {'precision': 0.9913520117021105, 'recall': 0.9937480865599975, 'f1-score': 0.9925486030642462, 'support': 62061.0}, 'eval_ADJ': {'precision': 0.9690797320855125, 'recall': 0.9712481608631682, 'f1-score': 0.9701627348019779, 'support': 32624.0}, 'eval_ADP': {'precision': 0.992139053459047, 'recall': 0.9929860181809792, 'f1-score': 0.9925623551388937, 'support': 43342.0}, 'eval_PRON': {'precision': 0.9931959175505303, 'recall': 0.9947885347765083, 'f1-score': 0.993991588223513, 'support': 14967.0}, 'eval_X': {'precision': 0.9120111731843575, 'recall': 0.6954206602768903, 'f1-score': 0.7891238670694865, 'support': 939.0}, 'eval_NUM': {'precision': 0.9828890489913544, 'recall': 0.992903930

100%|██████████| 15054/15054 [34:24<00:00,  7.29it/s]

{'train_runtime': 2064.0528, 'train_samples_per_second': 58.339, 'train_steps_per_second': 7.293, 'train_loss': 0.040027938208067514, 'epoch': 3.0}


TrainOutput(global_step=15054, training_loss=0.040027938208067514, metrics={'train_runtime': 2064.0528, 'train_samples_per_second': 58.339, 'train_steps_per_second': 7.293, 'train_loss': 0.040027938208067514, 'epoch': 3.0})

In [25]:
trainer.save_model('my_model')

In [26]:
pipe = pipeline(
    task='token-classification',
    model='my_model',
    device=device,
)

In [27]:
sentence = 'Need For Speed: Most Wanted is the best video game of all time.'
pipe(sentence)

[{'entity': 'NOUN',
  'score': 0.99942374,
  'index': 1,
  'word': 'Need',
  'start': 0,
  'end': 4},
 {'entity': 'ADP',
  'score': 0.9993299,
  'index': 2,
  'word': 'For',
  'start': 5,
  'end': 8},
 {'entity': 'NOUN',
  'score': 0.9998654,
  'index': 3,
  'word': 'Speed',
  'start': 9,
  'end': 14},
 {'entity': 'NOUN',
  'score': 0.63851696,
  'index': 4,
  'word': ':',
  'start': 14,
  'end': 15},
 {'entity': 'ADV',
  'score': 0.96408814,
  'index': 5,
  'word': 'Most',
  'start': 16,
  'end': 20},
 {'entity': 'VERB',
  'score': 0.91693956,
  'index': 6,
  'word': 'Wanted',
  'start': 21,
  'end': 27},
 {'entity': 'VERB',
  'score': 0.9999552,
  'index': 7,
  'word': 'is',
  'start': 28,
  'end': 30},
 {'entity': 'DET',
  'score': 0.9999356,
  'index': 8,
  'word': 'the',
  'start': 31,
  'end': 34},
 {'entity': 'ADJ',
  'score': 0.9996879,
  'index': 9,
  'word': 'best',
  'start': 35,
  'end': 39},
 {'entity': 'NOUN',
  'score': 0.997012,
  'index': 10,
  'word': 'video',
  'star